In [153]:
import numpy as np
import pandas as pd

In [154]:
train_label = pd.read_table('20news-bydate/matlab/train.label', names=['label'])
pi = []
log_pi = []
for i in range(20):
    pi_class = (sum(train_label['label']==i+1)+0.0)/len(train_label)
    pi.append(pi_class)
    log_pi.append(np.log(pi_class))


In [155]:
train_data = pd.read_csv('20news-bydate/matlab/train.data', delim_whitespace=True, names=['docidx', 'wordidx', 'count'])

In [156]:
vocabulary = pd.read_csv('vocabulary.txt',names=['vocabulary'])
vocabulary['wordidx'] = pd.DataFrame(range(1,len(vocabulary)+1,1))

In [157]:
index = 0
final_train_data = pd.DataFrame(columns=['wordidx','count','vocabulary','label'])
for i in range(20):
    to = len(train_label[train_label.label==i+1])
    label_i = train_data['docidx'].unique()[index:index+to]
    merged_train_data = train_data[train_data['docidx'].isin(label_i)].groupby(['wordidx'], as_index=False)['count'].sum()
    merged_train_data = merged_train_data.join(vocabulary.set_index('wordidx'),on='wordidx', how='outer',sort=True).fillna(0)
    merged_train_data['count'] = merged_train_data['count'] + 1 #+1 for laplace smoothing
    merged_train_data['label'] = i+1
    final_train_data = final_train_data.append(merged_train_data, ignore_index=True)
    index = index+to 

In [158]:
#model
total_count = final_train_data.groupby(['label'], as_index=False)['count'].sum()
model_train = final_train_data.merge(total_count, left_on='label',right_on='label', how='outer')
model_train['Pjw'] = model_train['count_x']/model_train['count_y']
model_train['log_Pjw'] = np.log(model_train['Pjw'])
model = model_train[['label', 'wordidx', 'Pjw','log_Pjw']]

In [159]:
model

,label,wordidx,Pjw,log_Pjw
0,1,1,0.000067,-9.615805
1,1,2,0.000305,-8.095980
2,1,3,0.001314,-6.634462
3,1,4,0.000048,-9.952278
4,1,5,0.000395,-7.836022
5,1,6,0.000200,-8.517193
6,1,7,0.000033,-10.308953
7,1,8,0.000010,-11.561716
8,1,9,0.000167,-8.699515
9,1,10,0.000671,-7.306103


In [160]:
test_data = pd.read_csv('20news-bydate/matlab/test.data', delim_whitespace=True, names=['docidx', 'wordidx', 'count'])

In [161]:
test_label = pd.read_table('20news-bydate/matlab/test.label', names=['test_label'])

In [162]:
for i in range(1,21,1):
    a = model[model.label==i].set_index('wordidx')['log_Pjw'] * test_data.groupby(['docidx','wordidx'])['count'].sum()
    a = a.reset_index(name=i)
    a = (a.groupby('docidx')[i].sum()+log_pi[i-1]).reset_index()
    if i != 1:
        b[i] = a[i]
    else:
        b = a

b = b.groupby('docidx').idxmax(axis=1).reset_index(name='estimated_label')
text_classification = b[['docidx','estimated_label']]

In [163]:
text_classification

,docidx,estimated_label
0,1,1
1,2,1
2,3,1
3,4,16
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


In [164]:
Error_rate = ((len(test_label)-sum(text_classification['estimated_label']==test_label['test_label'])+0.0)/len(test_label))*100

In [165]:
print 'Error_rate = ',Error_rate,"%"

Error_rate =  21.892071952 %
